In [1]:
import sys
sys.path.insert(0, '../src/libs')

In [2]:
# our file imports
from music_transformer import *
from transformer_training_helpers import *
from song_classification import *

# musicautobot
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.utils.midifile import *
from musicautobot.utils.file_processing import process_all

In [10]:
def topk_decode(model, src, src_mask, max_len, start_symbol, k):
  memory = model.encode(src, src_mask)
  ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
  for i in range(max_len-1):
    out, w2 = model.decode(memory, src_mask,
                       Variable(ys),
                       Variable(subsequent_mask(ys.size(1))
                                .type_as(src.data)))
    prob = model.generator(out[:, -1])
    _, next_words = torch.topk(prob, k = k, dim = 1)
    # if starting, then create 
    #print(next_words)
    if i == 0:
      next_word = next_words[0][0].data.item()
    else:
      k2 = random.randint(0,k-1)
      next_word = next_words[0][k2].data.item()
    #print(next_word)
    #k = random.randint(0,4)
    #next_word = next_words[k].data[0]
    ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
  
  return ys

def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out, w2 = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1) # change this for top k
        next_word = next_word.data[0]
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

Load model from previous state

In [5]:
model = torch.load('../models/epochs_30_LM.pt')

Load data

In [8]:
data_path = './saved_data/'
data_save_name = 'music_item_data_0-10000.pkl'
numpy_path = '../numpy_path'

numpy_files = get_files(numpy_path, extensions='.npy', recurse=True); len(numpy_files)

def create_databunch(files, path):
    #save_file.parent.mkdir(exist_ok=True, parents=True)
    vocab = MusicVocab.create()
    processors = [OpenNPFileProcessor(), MusicItemProcessor()]

    #data = MusicDataBunch.from_files(midi_files, path, processors=processors, encode_position=True)
    data = MusicDataBunch.from_files(files, path, processors=processors, encode_position=True)
    return data

all_data = create_databunch(numpy_files, data_path); all_data

C:\Users\Trevi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\fastai\core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


MusicDataBunch;

Train: LabelList (1060 items)
x: MusicItemList

MusicItem - (9818,)
xxbos xxpad n54 d2 n50 d2 n38 d1 xxsep d1...,
MusicItem - (6616,)
xxbos xxpad n64 d2 n61 d2 n57 d2 n52 d4...,
MusicItem - (9758,)
xxbos xxpad xxsep d7 n54 d11 xxsep d1 n64 d11...,
MusicItem - (1236,)
xxbos xxpad n76 d32 n72 d32 n67 d32 n64 d32...,
MusicItem - (7064,)
xxbos xxpad n76 d2 n72 d2 n60 d29 n57 d28...
y: LMLabelList
,,,,
Path: saved_data;

Valid: LabelList (117 items)
x: MusicItemList

MusicItem - (11442,)
xxbos xxpad xxsep d9 n55 d1 xxsep d1 n57 d1...,
MusicItem - (15824,)
xxbos xxpad xxsep d11 n89 d1 n88 d1 xxsep d1...,
MusicItem - (2552,)
xxbos xxpad n70 d10 n67 d10 n62 d10 n58 d10...,
MusicItem - (5066,)
xxbos xxpad n58 d1 n55 d1 n51 d1 n39 d4...,
MusicItem - (5798,)
xxbos xxpad n60 d128 n55 d32 n51 d32 n48 d12...
y: LMLabelList
,,,,
Path: saved_data;

Test: None

Generate topk/greedy outputs from the model

In [11]:
model.eval()
# ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
#src = Variable(torch.LongTensor([[1,2,3,4,5,6,7,8,9,10]]))
#src = Variable(torch.LongTensor([[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]])).cuda()
src = all_data.valid_ds[1][0].to_tensor()[:100].unsqueeze(dim=0)
#print(src.size())
src_mask = Variable(torch.ones(1, 1, 100)).unsqueeze(dim=0).cuda() # 26 was 10
output = greedy_decode(model.cuda(), src, src_mask, max_len=150, start_symbol=0)
output2 = topk_decode(model.cuda(), src, src_mask, max_len=150, start_symbol=0, k=2)

#print(output2[0])
#print(output[0])

output.cuda()
output2.cuda()

print("GREEDY: ", output[0])
print("TOPK: ", output2[0])
print("SRC: ", src[0])

ncorrect = 0
for i in range(len(src)):
  if src[0][i] == output2[0][i]:
    ncorrect += 1
print("TOPK: ", ncorrect / len(src[0]))

ncorrect = 0
for i in range(len(src)):
  if src[0][i] == output[0][i]:
    ncorrect += 1
print("GREEDY: ", ncorrect / len(src[0]))

GREEDY:  tensor([  0,   1,   8, 145,  38, 139,   8, 139,  38, 139,   8, 139,  38, 139,
          8, 139,  38, 139,   8, 139,  38, 139,   8, 139,  38, 139,   8, 139,
         38, 139,   8, 139,  38, 139,   8, 139,  38, 139,   8, 139,  38, 139,
          8, 139,  38, 139,   8, 139,  38, 139,   8, 139,  38, 139,   8, 139,
         38, 139,   8, 139,  38, 139,   8, 139,  38, 139,   8, 139,  38, 139,
          8, 139,  38, 139,   8, 139,  38, 139,   8, 139,   8, 139,  38, 139,
          8, 139,  81, 153,  78, 153,  74, 153,  69, 153,  66, 153,  62, 153,
         38, 139,   8, 139,  38, 139,   8, 139,  38, 139,   8, 139,  38, 139,
          8, 139,  38, 139,   8, 139,  38, 139,   8, 139,  38, 139,   8, 139,
         38, 139,   8, 139,  38, 139,   8, 139,  38, 139,   8, 139,  38, 139,
          8, 139,  38, 139,   8, 139,  38, 139,   8, 139], device='cuda:0')
TOPK:  tensor([  0,   1,  38, 138,   8, 138,  38, 139,   8, 139,  50, 138,  38, 139,
          8, 139,  50, 138,  38, 138,  38, 139,  3

Convert outputs to .mid

In [15]:
%matplotlib inline
from music21 import *
import fluidsynth
#environment.set()
#print(output)

#print(src.cpu().numpy())
MI = all_data.vocab.to_music_item(src.cpu().numpy()[0])
MO = all_data.vocab.to_music_item(output.cpu().numpy()[0])
MT = all_data.vocab.to_music_item(output2.cpu().numpy()[0])
print("SRC.SHP: ", src.shape)
print("OUTPUT.SHP: ", output.shape)
print("OUTPUT2.SHP: ", output2.shape)
print(output2)
#print(MI.to_text())
#print(MO.to_text())
MI.stream.write('midi', fp='./temp_outputs/inp.mid')
MO.stream.write('midi', fp='./temp_outputs/pred.mid')
MT.stream.write('midi', fp='./temp_outputs/topk.mid')
#path = os.path.join('./','inp.mid')
#print_play_mid('./inp.mid')
#print_mid('./inp.mid')
#print_mid('./pred.mid')
#print_mid('./topk.mid')

SRC.SHP:  torch.Size([1, 100])
OUTPUT.SHP:  torch.Size([1, 150])
OUTPUT2.SHP:  torch.Size([1, 150])
tensor([[  0,   1,  38, 138,   8, 138,  38, 139,   8, 139,  50, 138,  38, 139,
           8, 139,  50, 138,  38, 138,  38, 139,  38, 139,  38, 138,  38, 138,
           8, 138,  50, 138,   8, 139,  50, 138,  38, 138,   8, 139,  50, 138,
           8, 139,  38, 138,   8, 139,  38, 139,  38, 138,  38, 138,   8, 139,
          50, 138,   8, 139,  38, 138,  38, 139,   8, 139,  50, 138,   8, 138,
          50, 138,  38, 138,  38, 139,  38, 138,  38, 138,  38, 140,   8, 140,
          50, 140,   8, 140,  50, 138,  38, 140,   8, 138,  50, 139,  38, 139,
          38, 138,   8, 139,  50, 138,  38, 139,  38, 139,   8, 138,  81, 153,
           8, 140,  50, 139,   8, 140,  38, 139,   8, 139,  38, 139,   8, 140,
          38, 138,  38, 139,  38, 139,   8, 140,  38, 138,  38, 139,  38, 138,
          38, 138,   8, 139,  50, 138,   8, 138,  38, 139]], device='cuda:0')


'./temp_outputs/topk.mid'

Classify generated song's musical key

In [16]:
src_mitem = MusicItem.from_file('./temp_outputs/inp.mid', MusicVocab.create())
m_item = MusicItem.from_file('./temp_outputs/topk.mid', MusicVocab.create())
m_item2 = MusicItem.from_file('./temp_outputs/pred.mid', MusicVocab.create())
print(src_mitem.to_text())
print(int_to_note(34))
print(calculate_score(src_mitem))
print(calculate_score(m_item))
print(calculate_score(m_item2))

xxbos xxpad xxsep d11 n89 d1 n88 d1 xxsep d1 n86 d1 n84 d1 n83 d1 xxsep d1 n81 d1 n79 d1 n77 d1 xxsep d1 n76 d1 n74 d1 n72 d1 xxsep d1 n71 d1 n69 d1 n67 d1 xxsep d1 n77 d2 n69 d2 n65 d2 n62 d2 n58 d2 n46 d16 n34 d16 xxsep d1 n69 d2 n65 d2 n62 d2 n58 d2 xxsep d1 n69 d2 n65 d2 n62 d2 n58 d2 xxsep d1 n77 d2 n69 d2 n65 d2 n62 d2 n58 d2 xxsep d27 n46 d14 n34 d14
A#|Bb
('F_major', Counter({'F': 8, 'A#|Bb': 8, 'D': 6, 'A': 6, 'E': 2, 'C': 2, 'B': 2, 'G': 2}))
('F_major', Counter({'F': 31, 'C': 2}))
('F_major', Counter({'F': 37, 'C': 4, 'A': 4}))
